In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import h5py
import copy
import datetime
import ta
# Do you wanna see?
verbose = True

import sys
sys.path.append('../')
#%cd ..

from utils.data_extraction import load_data
from utils.data_cleaning import HFDataCleaning
#from .utils.data_extraction import load_data
#from .utils.data_cleaning import HFDataCleaning

### Load data etc

In [5]:
# print(os.listdir())
path = 'T:/taqhdf5'  #'a:/taqhdf5'
# allFiles = os.listdir(path)
# print(len(allFiles), allFiles[:5], allFiles[-5:])
# print(allFiles[-10:])

#dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,32)]).astype(int)
dates = np.array(['20200401']).astype(int)

# Provide a list of tickers of interest
tickers = ['GOOG']#'MSFT'

# Do we need data on trades, quotes or both?
dataNeeded = 'trades' # 'trades', 'quotes' or 'both'

# run load_data()
if dataNeeded == 'trades':
    tradeData = load_data(dates, tickers, dataNeeded, path, verbose)
elif dataNeeded == 'quotes':
    quoteData = load_data(dates, tickers, dataNeeded, path, verbose)
elif dataNeeded == 'both':
    tradeData, quoteData = load_data(dates, tickers, dataNeeded, path, verbose)

['.ipynb_checkpoints', 'testing_and_comparing_candlesticks.ipynb']
8515 ['taq_19930315.h5', 'taq_19930104.h5', 'taq_19930317.h5', 'taq_19930105.h5', 'taq_19930316.h5'] ['taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
['taqquote_20200515.h5', 'taqquote_20200518.h5', 'taqquote_20200519.h5', 'taqquote_20200520.h5', 'taqquote_20200521.h5', 'taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
##### Date range #####

Date, Min: 20200401
Date, Max: 20200401

##### Data Extraction begins #####

trade data is being extracted..

### Trade Data ###

The raw H5 trade file contains:  ['TradeIndex', 'Trades'] 

Ticker Information:  (b'GOOG            ', 26900500, 71427) 

Sneak peak of the data

            utcsec ex  cond  volume    price TradeStopStockIndicator corr  \
0  14400048517953  P  @ TI      67  1139.44                           00   
1  14422296771981

In [6]:
cleanedData = HFDataCleaning(['P1','p2','t1','p3'],tradeData,'trade',['q'])

In [73]:
step = 1/10 # in minutes

aggregateMinute = np.arange(0,60,step)
aggregateHour = np.arange(9,16,1)
aggregateDate = np.arange(len(cleanedData.Date.unique()))

remove = 30//step

# candle = np.zeros(((len(aggregateDate)*len(aggregateMinute)*len(aggregateHour)),4))
candleNP = np.zeros((((len(aggregateDate)*len(aggregateMinute)*len(aggregateHour))-remove*len(aggregateDate)),4))

TypeError: 'float' object cannot be interpreted as an integer

In [10]:
numpiedData = cleanedData[['Date','Hour','Minute']].to_numpy()
numpiedData = numpiedData.T
numpiedPrice = cleanedData['price'].to_numpy()

### Original

In [11]:


def candleCreateNP():
    ii = 0
    for l in cleanedData.Date.unique():
        for i in aggregateHour:
            for j in aggregateMinute:
                if (i == 9) & (j <30):
                    continue
                
                p1 = numpiedPrice[((numpiedData[0]==l)&\
                                     (numpiedData[1]==i)&\
                                     (numpiedData[2]>=j))&((numpiedData[0]==l)&\
                                                           (numpiedData[1]==i)&\
                                                           (numpiedData[2]<j+step))]
                if len(p1) > 0:
                    candleNP[ii] = np.array([p1[0],p1.max(),p1.min(),p1[-1]])
                else:
                    # if no new prices in the interval considered, use the previous pne
                    candleNP[ii] = candleNP[ii-1]
                ii += 1
                
    return candleNP

In [12]:
# Aggregate data in candle sticks
candleNP = candleCreateNP()

candleNP.shape

(390, 4)

In [24]:
candleNP[:5], candleNP[-5:]

(array([[1122.26, 1127.22, 1119.51, 1126.12],
        [1127.57, 1128.  , 1120.  , 1126.92],
        [1127.19, 1127.4 , 1124.02, 1125.44],
        [1125.01, 1125.19, 1122.97, 1123.  ],
        [1123.67, 1123.68, 1120.01, 1121.21]]),
 array([[1108.715, 1110.97 , 1108.715, 1110.97 ],
        [1110.95 , 1110.95 , 1104.1  , 1106.47 ],
        [1106.5  , 1106.9  , 1103.24 , 1106.05 ],
        [1105.65 , 1107.06 , 1105.17 , 1106.01 ],
        [1105.48 , 1107.3  , 1105.   , 1105.62 ]]))

### Attempt to vectorize

In [15]:
cleanedData['hour_min_col'] = cleanedData['Hour'] + cleanedData['Minute']/60
cleanedData.head()

,utcsec,ex,cond,volume,price,TradeStopStockIndicator,corr,TradeSequenceNumber,TradeID,SourceOfTrade,...,ParticipantTime,TRFTime,TTE,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,hour_min_col
0,34200012706854,Q,@ TI,83,1122.26,,00,36579,447,N,...,34200012683939,99,0,20200401,2020-04-01 09:30:00.012706,854,9,30,GOOG,9.5
1,34200012711048,Q,@ TI,17,1122.26,,00,36580,448,N,...,34200012683939,99,0,20200401,2020-04-01 09:30:00.012711,048,9,30,GOOG,9.5
2,34200012771200,Q,@ TI,83,1122.26,,00,36583,449,N,...,34200012747399,99,0,20200401,2020-04-01 09:30:00.012771,200,9,30,GOOG,9.5
3,34200012827903,Q,@ TI,83,1122.26,,00,36584,450,N,...,34200012803083,99,0,20200401,2020-04-01 09:30:00.012827,903,9,30,GOOG,9.5
4,34200013120215,Q,@ TI,20,1122.08,,00,36589,451,N,...,34200013098975,99,0,20200401,2020-04-01 09:30:00.013120,215,9,30,GOOG,9.5


In [49]:
cleanedData['hour_min_col'][(cleanedData['hour_min_col'] > 9.5) & (cleanedData['hour_min_col'] <= 9.5167)]

578    9.516667
579    9.516667
580    9.516667
581    9.516667
582    9.516667
583    9.516667
584    9.516667
585    9.516667
586    9.516667
587    9.516667
588    9.516667
589    9.516667
590    9.516667
591    9.516667
592    9.516667
593    9.516667
594    9.516667
595    9.516667
596    9.516667
597    9.516667
598    9.516667
599    9.516667
600    9.516667
601    9.516667
602    9.516667
603    9.516667
604    9.516667
605    9.516667
606    9.516667
607    9.516667
         ...   
760    9.516667
761    9.516667
762    9.516667
763    9.516667
764    9.516667
765    9.516667
766    9.516667
767    9.516667
768    9.516667
769    9.516667
770    9.516667
771    9.516667
772    9.516667
773    9.516667
774    9.516667
775    9.516667
776    9.516667
777    9.516667
778    9.516667
779    9.516667
780    9.516667
781    9.516667
782    9.516667
783    9.516667
784    9.516667
785    9.516667
786    9.516667
787    9.516667
788    9.516667
789    9.516667
Name: hour_min_col, Leng

In [16]:
cleanedData['hour_min_col'].min(), cleanedData['hour_min_col'].max()

(9.5, 15.983333333333333)

In [63]:
time_bins = np.arange(9.5-step/60,16+step/60,step/60)
time_bins, len(time_bins) 

(array([ 9.48333333,  9.5       ,  9.51666667,  9.53333333,  9.55      ,
         9.56666667,  9.58333333,  9.6       ,  9.61666667,  9.63333333,
         9.65      ,  9.66666667,  9.68333333,  9.7       ,  9.71666667,
         9.73333333,  9.75      ,  9.76666667,  9.78333333,  9.8       ,
         9.81666667,  9.83333333,  9.85      ,  9.86666667,  9.88333333,
         9.9       ,  9.91666667,  9.93333333,  9.95      ,  9.96666667,
         9.98333333, 10.        , 10.01666667, 10.03333333, 10.05      ,
        10.06666667, 10.08333333, 10.1       , 10.11666667, 10.13333333,
        10.15      , 10.16666667, 10.18333333, 10.2       , 10.21666667,
        10.23333333, 10.25      , 10.26666667, 10.28333333, 10.3       ,
        10.31666667, 10.33333333, 10.35      , 10.36666667, 10.38333333,
        10.4       , 10.41666667, 10.43333333, 10.45      , 10.46666667,
        10.48333333, 10.5       , 10.51666667, 10.53333333, 10.55      ,
        10.56666667, 10.58333333, 10.6       , 10.6

In [83]:
test_hour_min_col = pd.DataFrame([1,2,4,5])
test_hour_min_col
pd.cut(test_hour_min_col[0], bins=[1,2,3,4,5,6], right=False, labels=False)
#--> so cut() actually properly skips a bin integer if there is no data in it

0    0
1    1
2    3
3    4
Name: 0, dtype: int64

In [87]:
np.isin([0,1,2,3,4], pd.cut(test_hour_min_col[0], bins=[1,2,3,4,5,6], right=False, labels=False).values)

array([ True,  True, False,  True,  True])

In [97]:
np.arange(len(time_bins[1:-1]))

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [107]:
cut_test = pd.cut(cleanedData['hour_min_col'], bins=time_bins, right=True, labels=False).values
cut_test = cut_test[cut_test != 2]
cut_test

array([  0,   0,   0, ..., 389, 389, 389], dtype=int64)

In [109]:
#np.isin(np.arange(len(time_bins[1:-1])), pd.cut(cleanedData['hour_min_col'], bins=time_bins, right=True, labels=False).values)
isin_cut = np.isin(np.arange(len(time_bins[1:-1])), cut_test)

np.where(isin_cut==False)

# create temporary version of cleanedData and add new observation with date, time_group, t-1 price for each missing. Sweep this in a while loop


(array([2], dtype=int64),)

In [110]:
cleanedData['hour_min_col']

0         9.500000
1         9.500000
2         9.500000
3         9.500000
4         9.500000
5         9.500000
6         9.500000
7         9.500000
8         9.500000
9         9.500000
10        9.500000
11        9.500000
12        9.500000
13        9.500000
14        9.500000
15        9.500000
16        9.500000
17        9.500000
18        9.500000
19        9.500000
20        9.500000
21        9.500000
22        9.500000
23        9.500000
24        9.500000
25        9.500000
26        9.500000
27        9.500000
28        9.500000
29        9.500000
           ...    
23332    15.983333
23333    15.983333
23334    15.983333
23335    15.983333
23336    15.983333
23337    15.983333
23338    15.983333
23339    15.983333
23340    15.983333
23341    15.983333
23342    15.983333
23343    15.983333
23344    15.983333
23345    15.983333
23346    15.983333
23347    15.983333
23348    15.983333
23349    15.983333
23350    15.983333
23351    15.983333
23352    15.983333
23353    15.

In [91]:
pd.cut(cleanedData['hour_min_col'], bins=time_bins, right=True, labels=False)

0          0
1          0
2          0
3          0
4          0
5          0
6          0
7          0
8          0
9          0
10         0
11         0
12         0
13         0
14         0
15         0
16         0
17         0
18         0
19         0
20         0
21         0
22         0
23         0
24         0
25         0
26         0
27         0
28         0
29         0
        ... 
23332    389
23333    389
23334    389
23335    389
23336    389
23337    389
23338    389
23339    389
23340    389
23341    389
23342    389
23343    389
23344    389
23345    389
23346    389
23347    389
23348    389
23349    389
23350    389
23351    389
23352    389
23353    389
23354    389
23355    389
23356    389
23357    389
23358    389
23359    389
23360    389
23361    389
Name: hour_min_col, Length: 23362, dtype: int64

In [112]:
cleanedData['time_group'] = pd.cut(cleanedData['hour_min_col'], bins=time_bins, right=True, labels=False) #0#+1
cleanedData

,utcsec,ex,cond,volume,price,TradeStopStockIndicator,corr,TradeSequenceNumber,TradeID,SourceOfTrade,...,TRFTime,TTE,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,hour_min_col,time_group
0,34200012706854,Q,@ TI,83,1122.26,,00,36579,447,N,...,99,0,20200401,2020-04-01 09:30:00.012706,854,9,30,GOOG,9.500000,0
1,34200012711048,Q,@ TI,17,1122.26,,00,36580,448,N,...,99,0,20200401,2020-04-01 09:30:00.012711,048,9,30,GOOG,9.500000,0
2,34200012771200,Q,@ TI,83,1122.26,,00,36583,449,N,...,99,0,20200401,2020-04-01 09:30:00.012771,200,9,30,GOOG,9.500000,0
3,34200012827903,Q,@ TI,83,1122.26,,00,36584,450,N,...,99,0,20200401,2020-04-01 09:30:00.012827,903,9,30,GOOG,9.500000,0
4,34200013120215,Q,@ TI,20,1122.08,,00,36589,451,N,...,99,0,20200401,2020-04-01 09:30:00.013120,215,9,30,GOOG,9.500000,0
5,34200134121465,Q,@ TI,1,1122.25,,00,36816,452,N,...,99,0,20200401,2020-04-01 09:30:00.134121,465,9,30,GOOG,9.500000,0
6,34200137711405,Q,@ TI,1,1121.00,,00,36821,453,N,...,99,0,20200401,2020-04-01 09:30:00.137711,405,9,30,GOOG,9.500000,0
7,34200194838863,Q,@FTI,38,1122.26,,00,36870,454,N,...,99,1,20200401,2020-04-01 09:30:00.194838,863,9,30,GOOG,9.500000,0
8,34200194842233,Q,@FTI,1,1122.26,,00,36871,455,N,...,99,1,20200401,2020-04-01 09:30:00.194842,233,9,30,GOOG,9.500000,0
9,34200194877651,Q,@FTI,15,1124.00,,00,36872,456,N,...,99,1,20200401,2020-04-01 09:30:00.194877,651,9,30,GOOG,9.500000,0


In [113]:
# check for missing time_group
np.unique(cleanedData['time_group'])

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [66]:
cleanedData[(cleanedData['hour_min_col'] > 9.6) & (cleanedData['hour_min_col'] < 9.7)]

,utcsec,ex,cond,volume,price,TradeStopStockIndicator,corr,TradeSequenceNumber,TradeID,SourceOfTrade,...,TRFTime,TTE,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,hour_min_col,time_group
1774,34621200075630,Q,@F I,6,1124.98,,00,157628,2221,N,...,99,1,20200401,2020-04-01 09:37:01.200075,630,9,37,GOOG,9.616667,7
1775,34621200080193,Q,@F I,10,1125.00,,00,157629,2222,N,...,99,1,20200401,2020-04-01 09:37:01.200080,193,9,37,GOOG,9.616667,7
1776,34621200192318,Q,@F I,5,1125.00,,00,157630,2223,N,...,99,1,20200401,2020-04-01 09:37:01.200192,318,9,37,GOOG,9.616667,7
1777,34621203396369,Q,@F I,13,1125.52,,00,157653,2224,N,...,99,1,20200401,2020-04-01 09:37:01.203396,369,9,37,GOOG,9.616667,7
1778,34621204241407,Q,@F I,7,1125.52,,00,157655,2225,N,...,99,1,20200401,2020-04-01 09:37:01.204241,407,9,37,GOOG,9.616667,7
1779,34624846143673,Q,@F I,20,1126.00,,00,158379,2226,N,...,99,1,20200401,2020-04-01 09:37:04.846143,673,9,37,GOOG,9.616667,7
1780,34624846147812,Q,@F I,1,1126.00,,00,158380,2227,N,...,99,1,20200401,2020-04-01 09:37:04.846147,812,9,37,GOOG,9.616667,7
1781,34624847803554,Q,@F I,1,1126.41,,00,158400,2228,N,...,99,1,20200401,2020-04-01 09:37:04.847803,554,9,37,GOOG,9.616667,7
1782,34624849420780,Q,@F I,9,1126.50,,00,158413,2229,N,...,99,1,20200401,2020-04-01 09:37:04.849420,780,9,37,GOOG,9.616667,7
1783,34624849670690,Q,@ I,14,1126.50,,00,158414,2230,N,...,99,0,20200401,2020-04-01 09:37:04.849670,690,9,37,GOOG,9.616667,7


In [67]:
cleanedData.groupby(['Date','time_group'])[['price','Timestamp']].last() # .last()

# or insert new observation here with the missing time_group

price                  Timestamp
Date     time_group                                    
20200401 0           1126.12 2020-04-01 09:30:58.001385
         1           1126.92 2020-04-01 09:31:59.029981
         2           1125.44 2020-04-01 09:32:59.161537
         3           1123.00 2020-04-01 09:33:54.125338
         4           1121.21 2020-04-01 09:34:58.749776
         5           1117.44 2020-04-01 09:35:57.004849
         6           1124.40 2020-04-01 09:36:59.589734
         7           1127.65 2020-04-01 09:37:58.782698
         8           1125.51 2020-04-01 09:38:57.695308
         9           1124.52 2020-04-01 09:39:56.779269
         10          1124.52 2020-04-01 09:40:53.347684
         11          1125.77 2020-04-01 09:41:55.758752
         12          1125.35 2020-04-01 09:42:57.084996
         13          1125.18 2020-04-01 09:43:57.015881
         14          1124.76 2020-04-01 09:44:53.256203
         15          1122.50 2020-04-01 09:45:53.315858
         16          1118.26 2020-04-01 09:46:49.183536
         17          1119.07 2020-04-01 09:47:52.330064
         18          1119.77 2020-04-01 09:48:44.594046
         19          1120.29 2020-04-01 09:49:58.874800
         20          1120.75 2020-04-01 09:50:53.389917
         21          1120.19 2020-04-01 09:51:55.484069
         22          1117.61 2020-04-01 09:52:56.637265
         23          1120.93 2020-04-01 09:53:57.644114
         24          1118.92 2020-04-01 09:54:58.468796
         25          1118.57 2020-04-01 09:55:50.936623
         26          1118.55 2020-04-01 09:56:54.750055
         27          1118.66 2020-04-01 09:57:56.922049
         28          1115.30 2020-04-01 09:58:59.858874
         29          1115.55 2020-04-01 09:59:59.939460
...                      ...                        ...
         360         1100.12 2020-04-01 15:30:59.778439
         361         1101.28 2020-04-01 15:31:59.899195
         362         1100.69 2020-04-01 15:32:59.973326
         363         1100.81 2020-04-01 15:33:59.092286
         364         1099.36 2020-04-01 15:34:57.324475
         365         1099.42 2020-04-01 15:35:29.078376
         366         1099.42 2020-04-01 15:36:58.261454
         367         1100.90 2020-04-01 15:37:58.670363
         368         1099.11 2020-04-01 15:38:59.567366
         369         1097.88 2020-04-01 15:39:58.565954
         370         1099.74 2020-04-01 15:40:57.703773
         371         1100.48 2020-04-01 15:41:55.980874
         372         1099.48 2020-04-01 15:42:58.028081
         373         1100.15 2020-04-01 15:43:59.821423
         374         1101.45 2020-04-01 15:44:54.482108
         375         1105.71 2020-04-01 15:45:54.942925
         376         1105.83 2020-04-01 15:46:54.505842
         377         1105.82 2020-04-01 15:47:58.934373
         378         1109.43 2020-04-01 15:48:59.941288
         379         1106.87 2020-04-01 15:49:59.040758
         380         1103.30 2020-04-01 15:50:59.953214
         381         1105.96 2020-04-01 15:51:59.069931
         382         1104.08 2020-04-01 15:52:58.484496
         383         1106.97 2020-04-01 15:53:59.530913
         384         1108.85 2020-04-01 15:54:59.455595
         385         1110.97 2020-04-01 15:55:59.609969
         386         1106.47 2020-04-01 15:56:58.812399
         387         1106.05 2020-04-01 15:57:58.895350
         388         1106.01 2020-04-01 15:58:58.701608
         389         1105.62 2020-04-01 15:59:59.999297

[390 rows x 2 columns]

In [68]:
OHLC = cleanedData.groupby(['Date','time_group'])[['price']].agg(['first', 'max', 'min', 'last'])
OHLC.values.shape

(390, 4)

In [69]:
OHLC.values[:5], OHLC.values[-5:]

(array([[1122.26, 1127.22, 1119.51, 1126.12],
        [1127.57, 1128.  , 1120.  , 1126.92],
        [1127.19, 1127.4 , 1124.02, 1125.44],
        [1125.01, 1125.19, 1122.97, 1123.  ],
        [1123.67, 1123.68, 1120.01, 1121.21]]),
 array([[1108.715, 1110.97 , 1108.715, 1110.97 ],
        [1110.95 , 1110.95 , 1104.1  , 1106.47 ],
        [1106.5  , 1106.9  , 1103.24 , 1106.05 ],
        [1105.65 , 1107.06 , 1105.17 , 1106.01 ],
        [1105.48 , 1107.3  , 1105.   , 1105.62 ]]))

In [70]:
# set up as function
def candleCreateNP_vect(verbose=True):
        
    cleanedData['hour_min_col'] = cleanedData['Hour'] + cleanedData['Minute']/60
    if verbose:
        print(f"min and max of new hour_min_col: \
              {cleanedData['hour_min_col'].min()}, {cleanedData['hour_min_col'].max()}")
              
    # setup time_bins to group each timestamp
    delta = step/60
    time_bins = np.arange(9.5-delta, 16+delta, delta)
              
    # put each timestamp into a bucket according to time_bins defined by the step variable
    cleanedData['time_group'] = pd.cut(cleanedData['hour_min_col'], bins=time_bins, right=True, labels=False)
    
    # group by date and time_group, extract price, take it first, max, min, last (open, high, low, close)
    OHLC = cleanedData.groupby(['Date','time_group'])[['price']].agg(['first', 'max', 'min', 'last'])              
    
    # return as numpy if preferred
    return OHLC.values

### Compare timing once

In [71]:
import time
a = time.time()
candleNP1 = candleCreateNP()
b = time.time()
print(b-a)

a = time.time()
candleNP2 = candleCreateNP_vect()
b = time.time()
print(b-a)

1.4680838584899902
min and max of new hour_min_col:               9.5, 15.983333333333333
0.02800154685974121


In [72]:
# Make sure they are equal
(candleNP1.shape == candleNP2.shape), \
(candleNP1 == candleNP2).all(), \
np.array_equal(candleNP1, candleNP2)

(True, True, True)

### Multiple time-it runs

In [24]:
# Aggregate data in candle sticks
%timeit candleNP1 = candleCreateNP()
%timeit candleNP2 = candleCreateNP_vect(verbose=False)

56.8 s ± 498 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
132 ms ± 2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
